In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv("soy_real_text_train.txt",encoding="utf-8-sig")

In [3]:
df_train2 = pd.read_csv("train.csv",encoding="utf-8-sig") #label 따로

In [4]:
from eunjeon import Mecab
mecab = Mecab()

In [5]:
df_train["text_pos"] = df_train["text"].apply(lambda x : mecab.pos(x))

In [6]:
df_train

,index,text,text_pos
0,0,XXX 은행 성산 XXX 팀장 입니다 행복한 주말 되세요,"[(XXX, SL), (은행, NNG), (성산, NNG), (XXX, SL), (..."
1,1,오늘도 많이 웃으시는 하루 시작 하세요 XXX 은행 진월동 VIP 라운지 XXX 올림,"[(오늘, NNG), (도, JX), (많이, MAG), (웃, VV), (으시, ..."
2,2,안녕 하십니까 고객님 XXX 은행 입니다 금일 납부하셔야 할 금액 은 153600원...,"[(안녕, IC), (하, VV), (십니까, EP+EF), (고객, NNG), (..."
3,3,XXX 고객님 안녕 하세요 XXX 은행 XXX 지점 입니다 지난 한 해 동안 저희 ...,"[(XXX, SL), (고객, NNG), (님, XSN), (안녕, NNG), (하..."
4,4,1월은 새로움이 가득XXX 입니다 올 한해 더 많이 행복한 한해되시길 바랍니다,"[(1, SN), (월, NNBC), (은, JX), (새로움, VA+ETN), (..."
...,...,...,...
295940,295940,XXX 고객님 열심히 달려왔던 2018년도 어느 새 뒤안길로 지나쳐가고 벅찬 설렘으...,"[(XXX, SL), (고객, NNG), (님, XSN), (열심히, MAG), (..."
295941,295941,XXX 고객님 실버웰빙신탁이 만기 도래 예정 입니다 XXX 남양주지점,"[(XXX, SL), (고객, NNG), (님, XSN), (실버, NNP), (웰..."
295942,295942,한해동안 XXX 은행 과 함께 해주셔서 정말 감사드립니다 2019년 기해년 을 맞이...,"[(한, MM), (해, NNG), (동안, NNG), (XXX, SL), (은행,..."
295943,295943,1228금 예금 및 적금 만기 입니다 예금 은 시간내서 내점하시고 적금은 1년 자동...,"[(1228, SN), (금, NNG), (예금, NNG), (및, MAJ), (적..."


In [7]:
POS_TAGGER = ['NNG','SL','MAG','VV','NNP','XSN','VV+EC','NNBC','NM','VX','SN','NNB','NNBC','NNG','VA','IC','MM']
def tagging(pos):
    word_list = []
    for x in pos:
        if x[1] in POS_TAGGER:
            word_list.append(x[0])
    return word_list

In [8]:
df_train["text_pic"] = df_train["text_pos"].apply(lambda x : tagging(x))

In [9]:
df_train

,index,text,text_pos,text_pic
0,0,XXX 은행 성산 XXX 팀장 입니다 행복한 주말 되세요,"[(XXX, SL), (은행, NNG), (성산, NNG), (XXX, SL), (...","[XXX, 은행, 성산, XXX, 팀장, 행복, 주말, 되]"
1,1,오늘도 많이 웃으시는 하루 시작 하세요 XXX 은행 진월동 VIP 라운지 XXX 올림,"[(오늘, NNG), (도, JX), (많이, MAG), (웃, VV), (으시, ...","[오늘, 많이, 웃, 하루, 시작, 하, XXX, 은행, 진월동, VIP, 라운지,..."
2,2,안녕 하십니까 고객님 XXX 은행 입니다 금일 납부하셔야 할 금액 은 153600원...,"[(안녕, IC), (하, VV), (십니까, EP+EF), (고객, NNG), (...","[안녕, 하, 고객, 님, XXX, 은행, 금일, 납부, 금액, 153600, 원,..."
3,3,XXX 고객님 안녕 하세요 XXX 은행 XXX 지점 입니다 지난 한 해 동안 저희 ...,"[(XXX, SL), (고객, NNG), (님, XSN), (안녕, NNG), (하...","[XXX, 고객, 님, 안녕, 하, XXX, 은행, XXX, 지점, 한, 해, 동안..."
4,4,1월은 새로움이 가득XXX 입니다 올 한해 더 많이 행복한 한해되시길 바랍니다,"[(1, SN), (월, NNBC), (은, JX), (새로움, VA+ETN), (...","[1, 월, 가득, XXX, 올, 한, 더, 많이, 행복, 한, 해, 바랍니다]"
...,...,...,...,...
295940,295940,XXX 고객님 열심히 달려왔던 2018년도 어느 새 뒤안길로 지나쳐가고 벅찬 설렘으...,"[(XXX, SL), (고객, NNG), (님, XSN), (열심히, MAG), (...","[XXX, 고객, 님, 열심히, 2018, 년, 어느, 새, 뒤안길, 지나쳐, 가,..."
295941,295941,XXX 고객님 실버웰빙신탁이 만기 도래 예정 입니다 XXX 남양주지점,"[(XXX, SL), (고객, NNG), (님, XSN), (실버, NNP), (웰...","[XXX, 고객, 님, 실버, 웰빙, 신탁, 만기, 도래, 예정, XXX, 남양주,..."
295942,295942,한해동안 XXX 은행 과 함께 해주셔서 정말 감사드립니다 2019년 기해년 을 맞이...,"[(한, MM), (해, NNG), (동안, NNG), (XXX, SL), (은행,...","[한, 해, 동안, XXX, 은행, 과, 함께, 해, 주, 정말, 감사, 드립니다,..."
295943,295943,1228금 예금 및 적금 만기 입니다 예금 은 시간내서 내점하시고 적금은 1년 자동...,"[(1228, SN), (금, NNG), (예금, NNG), (및, MAJ), (적...","[1228, 금, 예금, 적금, 만기, 예, 금, 시간, 내서, 내점, 적금, 1,..."


In [20]:
df_train.text_pic.to_csv("confirm_pos.csv",encoding="utf-8-sig")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [23]:
from sklearn.metrics import roc_auc_score
def auroc(y_true,y_pred):
    return tf.py_function(roc_auc_score,(y_true,y_pred),tf.double)

#def roc_auc_score_FIXED(y_true, y_pred):
#    if len(np.unique(y_true)) == 1: # bug in roc_auc_score
#        return accuracy_score(y_true, np.rint(y_pred))
#    return roc_auc_score(y_true, y_pred)

#roc_auc_score(y_true=y, y_score=, average='macro')
#from sklearn.metrics import roc_auc_score
#def auroc(y_true,y_pred):
#    return roc_auc_score(y_true, y_pred)

In [26]:
from keras.optimizers import Adam
adam = Adam(lr=0.00001)

In [18]:
from sklearn import metrics
from tensorflow import keras

In [40]:
max_features = 12000
tokenizer = Tokenizer(num_words=max_features) #번호를 매겨줌
tokenizer.fit_on_texts(df_train['text_pic'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['text_pic'])
maxlen = 300 #350이 전체 데이터 5% 손실에 해당, 직전 모델 : 120 (전체 데이터 20% 손실)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = df_train2['smishing']

In [41]:
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True))) # LSTM은 RNN의 특이한 종류 중 하나
model.add(GlobalMaxPool1D()) #풀링 : 과적합 방지
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05)) # 과적합 방지 0~1 사이, 작을수록 과적합 방지 0.05:5%만 학습
model.add(Dense(1, activation="sigmoid")) #다중분류
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',auroc]) #대중적인 optimizer

In [42]:
batch_size = 300 # 200개씩 학습 후 채점
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs)

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
295945/295945 [==============================] - 4243s 14ms/step - loss: 0.4093 - accuracy: 0.9269 - auroc: 0.8942
Epoch 2/3
295945/295945 [==============================] - 4407s 15ms/step - loss: 0.1732 - accuracy: 0.9455 - auroc: 0.9818
Epoch 3/3
295945/295945 [==============================] - 4450s 15ms/step - loss: 0.1011 - accuracy: 0.9575 - auroc: 0.9989


In [43]:
df_test = pd.read_csv("public_test.csv")

In [44]:
df_test["text_pos"] = df_test["text"].apply(lambda x : mecab.pos(x))
df_test["text_pic"] = df_test["text_pos"].apply(lambda x : tagging(x))

In [45]:
list_tokenized_test = tokenizer.texts_to_sequences(df_test["text_pic"])
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [46]:
prediction = model.predict_proba(X_te)

In [47]:
df_test["smishing"] = prediction

In [17]:
final = df_test[["id","smishing"]]
final

,id,smishing
0,340000,0.000007
1,340001,0.000058
2,340002,0.000012
3,340003,0.000003
4,340004,0.000010
...,...,...
1621,341621,0.000029
1622,341622,0.000025
1623,341623,0.000008
1624,341624,0.000032


In [32]:
final = df_test[["id","smishing"]]
final

,id,smishing
0,340000,1.192093e-07
1,340001,8.940697e-08
2,340002,0.000000e+00
3,340003,0.000000e+00
4,340004,0.000000e+00
...,...,...
1621,341621,8.940697e-08
1622,341622,0.000000e+00
1623,341623,0.000000e+00
1624,341624,3.261857e-08


In [42]:
final = df_test[["id","smishing"]]
final

,id,smishing
0,340000,3.874302e-07
1,340001,7.957220e-06
2,340002,1.490116e-07
3,340003,2.980232e-07
4,340004,1.192093e-07
...,...,...
1621,341621,8.046627e-07
1622,341622,1.788139e-07
1623,341623,1.490116e-07
1624,341624,8.330186e-07


In [43]:
final.to_csv("submission_embed_LSTM_initial_pos2.csv",encoding="utf-8-sig", index=False)

In [48]:
final = df_test[["id","smishing"]]
final

,id,smishing
0,340000,0.050362
1,340001,0.076413
2,340002,0.022947
3,340003,0.025333
4,340004,0.025969
...,...,...
1621,341621,0.025028
1622,341622,0.023702
1623,341623,0.022715
1624,341624,0.061345


In [49]:
final.to_csv("submission_embed_LSTM_initial_pos_auroc.csv",encoding="utf-8-sig", index=False)